In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.impute import SimpleImputer


In [2]:
d1=pd.read_csv("D:/Desktop/AD/AD_Dataset/Gene_expression/gene_451_last1_genes.csv")
d2=pd.read_csv("d:/Desktop/AD/AD_Dataset/DNA_methylation/dmp_last_419.csv")
data=pd.read_csv("D:/Desktop/AD/AD_Dataset/Gene_expression/GSE33000_filtered_file_orginal_AD.csv")
data1=pd.read_csv("d:/Desktop/AD/AD_Dataset/DNA_methylation/dmp_disease_status.csv")
data['disease_status'] = data['disease_status'].apply(lambda x: 1 if x == "Alzheimer's disease" else 0)

In [3]:
# Add first and last column from AD_gene to d1
d1 = pd.concat([data.iloc[:, [0, -1]], d1], axis=1)
d2 = pd.concat([data1.iloc[:,  -1], d2], axis=1)
d2['Disease State'] = d2['Disease State'].apply(lambda x: 1 if x == "Alzheimer's disease" else 0)
data1 = data1.applymap(lambda x: x.replace('"', '') if isinstance(x, str) else x)

In [4]:
# Get the gene names from both datasets (excluding Sample)
genes_d1 = set(d1.columns)-{"Sample","disease_status"}
genes_d2 = set(d2.columns) - {"Sample"}

# Find intersection
common_genes = genes_d1.intersection(genes_d2)

# Create new DataFrames containing only intersecting genes + Sample
d1_common=d1[["Sample","disease_status"] + list(common_genes)]
d2_common=d2[["Sample","Disease State"] + list(common_genes)]


In [5]:
d1_common.head()

,Sample,disease_status,DDX3Y,ZCCHC12,CNKSR2,SLC14A1,RBM3,SERPINA5,FRMPD4,BEX1,RPS4Y1,PAK3,BEX5,FGF13,GPRASP1,SYTL4,HPRT1,TMSB4Y,MS4A4A
0,GSM1423780,1,-1.418092,0.040549,-0.020435,0.127394,0.341209,0.233161,-0.044839,0.081533,-1.658940,-0.086508,-0.007898,0.019651,0.055005,-0.165616,0.136986,-0.953641,0.177249
1,GSM1423781,1,0.255052,-0.111576,0.072077,0.053273,0.146683,-0.295554,-0.033920,0.142487,0.225559,-0.069293,-0.036374,0.014815,0.002815,-0.172584,0.047247,0.232091,0.270190
2,GSM1423782,1,0.254411,0.154966,0.117780,-0.582372,0.157258,-0.598633,0.145267,0.225043,0.330395,0.142142,0.135785,0.150028,0.123408,-0.388533,0.160844,0.386145,-0.131901
3,GSM1423783,1,-1.470255,-0.135880,-0.173845,0.245438,0.162723,0.237143,-0.176507,0.052789,-1.563496,-0.251430,-0.010927,-0.055892,-0.047806,-0.087103,0.010585,-1.215171,0.101474
4,GSM1423784,1,-1.513204,-0.106902,0.057470,0.014268,-0.196429,-0.775671,0.087054,0.092603,-1.691422,0.116100,0.018407,-0.069487,0.073264,-0.354625,0.143043,-1.320286,-0.220523


In [6]:
common_genes

{'BEX1',
 'BEX5',
 'CNKSR2',
 'DDX3Y',
 'FGF13',
 'FRMPD4',
 'GPRASP1',
 'HPRT1',
 'MS4A4A',
 'PAK3',
 'RBM3',
 'RPS4Y1',
 'SERPINA5',
 'SLC14A1',
 'SYTL4',
 'TMSB4Y',
 'ZCCHC12'}

In [7]:
d2_common = d2_common.rename(columns={"Disease State": "disease_status"})

In [8]:
# Combine row-wise (stack samples)
merged_df = pd.concat([d1_common, d2_common], axis=0, ignore_index=True)

# Check result
merged_df.shape
merged_df.head()

,Sample,disease_status,DDX3Y,ZCCHC12,CNKSR2,SLC14A1,RBM3,SERPINA5,FRMPD4,BEX1,RPS4Y1,PAK3,BEX5,FGF13,GPRASP1,SYTL4,HPRT1,TMSB4Y,MS4A4A
0,GSM1423780,1,-1.418092,0.040549,-0.020435,0.127394,0.341209,0.233161,-0.044839,0.081533,-1.658940,-0.086508,-0.007898,0.019651,0.055005,-0.165616,0.136986,-0.953641,0.177249
1,GSM1423781,1,0.255052,-0.111576,0.072077,0.053273,0.146683,-0.295554,-0.033920,0.142487,0.225559,-0.069293,-0.036374,0.014815,0.002815,-0.172584,0.047247,0.232091,0.270190
2,GSM1423782,1,0.254411,0.154966,0.117780,-0.582372,0.157258,-0.598633,0.145267,0.225043,0.330395,0.142142,0.135785,0.150028,0.123408,-0.388533,0.160844,0.386145,-0.131901
3,GSM1423783,1,-1.470255,-0.135880,-0.173845,0.245438,0.162723,0.237143,-0.176507,0.052789,-1.563496,-0.251430,-0.010927,-0.055892,-0.047806,-0.087103,0.010585,-1.215171,0.101474
4,GSM1423784,1,-1.513204,-0.106902,0.057470,0.014268,-0.196429,-0.775671,0.087054,0.092603,-1.691422,0.116100,0.018407,-0.069487,0.073264,-0.354625,0.143043,-1.320286,-0.220523


In [ ]:
merged_df.to_csv("D:/Desktop/AD_Dataset/DNA_methylation/GSE33000_series_matrix.csv", index=False)


In [9]:
y=merged_df.iloc[:,1]
y.shape
x=merged_df.iloc[:,2:19]

In [10]:
x

,DDX3Y,ZCCHC12,CNKSR2,SLC14A1,RBM3,SERPINA5,FRMPD4,BEX1,RPS4Y1,PAK3,BEX5,FGF13,GPRASP1,SYTL4,HPRT1,TMSB4Y,MS4A4A
0,-1.418092,0.040549,-0.020435,0.127394,0.341209,0.233161,-0.044839,0.081533,-1.658940,-0.086508,-0.007898,0.019651,0.055005,-0.165616,0.136986,-0.953641,0.177249
1,0.255052,-0.111576,0.072077,0.053273,0.146683,-0.295554,-0.033920,0.142487,0.225559,-0.069293,-0.036374,0.014815,0.002815,-0.172584,0.047247,0.232091,0.270190
2,0.254411,0.154966,0.117780,-0.582372,0.157258,-0.598633,0.145267,0.225043,0.330395,0.142142,0.135785,0.150028,0.123408,-0.388533,0.160844,0.386145,-0.131901
3,-1.470255,-0.135880,-0.173845,0.245438,0.162723,0.237143,-0.176507,0.052789,-1.563496,-0.251430,-0.010927,-0.055892,-0.047806,-0.087103,0.010585,-1.215171,0.101474
4,-1.513204,-0.106902,0.057470,0.014268,-0.196429,-0.775671,0.087054,0.092603,-1.691422,0.116100,0.018407,-0.069487,0.073264,-0.354625,0.143043,-1.320286,-0.220523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,-0.769763,-1.757846,-0.545826,0.154342,-0.808145,0.406802,-1.314473,-0.907693,-0.904005,0.328266,-0.429961,-0.444084,-0.788910,-0.341702,-1.078059,-0.824792,-0.089591
749,-0.683578,-0.691529,-0.616416,0.036069,-0.847690,3.204724,-1.403329,-0.919475,-0.561736,0.183643,-0.392802,-0.307045,-0.734319,-0.312581,-1.478574,-0.646027,-3.427495
750,-0.783844,-1.105804,-0.769406,0.083931,-0.938876,-3.764702,-2.002255,-0.880671,-0.558501,-0.015639,-0.677495,-0.460924,-1.068545,-0.440326,-1.516296,-0.604200,-3.881480
751,2.658161,-3.788603,-3.317366,0.400682,-3.100065,3.494215,-4.734259,-3.040330,2.332619,-2.310084,-2.591065,-3.266104,-4.111160,-2.700314,-4.634969,2.217983,-0.334230


In [11]:
from sklearn.impute import SimpleImputer

# Handle missing values before scaling
imputer = SimpleImputer(strategy="mean")   # or "median"
x = imputer.fit_transform(x)


In [12]:
np.isnan(x).any()


False

In [13]:
scaler = StandardScaler()
x_scaled_array = scaler.fit_transform(x)

# # # Create scaled DataFrame with same column names and index
# x_scaled = pd.DataFrame(x_scaled_array, columns=x.columns, index=x.index)

# # # Check the result
# print(x_scaled.head())

In [14]:
# Handle missing values before scaling
imputer = SimpleImputer(strategy="mean")   # or "median"
X_imputed = imputer.fit_transform(x)
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# Suppose x is your NumPy array (753, 17)
print("Original shape:", x.shape)

poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
x_poly = poly.fit_transform(X_imputed)

print("With polynomial features:", x_poly.shape)

# Continue with scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x_poly)


# ----------------
# 3. Handle class imbalance
# ----------------
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

Original shape: (753, 17)
With polynomial features: (753, 170)


In [15]:
# Split into train/test set
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [16]:
# MLP Classifier
mlp = MLPClassifier(random_state=42, max_iter=1000)

# Grid Search parameters
param_grid = {
    'hidden_layer_sizes': [ (200,100,100,50,50)],
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.0001],
    'learning_rate': ['constant']
}

# GridSearchCV with 5-fold CV
grid_search = GridSearchCV(estimator=mlp,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
                           n_jobs=-1,
                           verbose=2)
# Fit the model
grid_search.fit(X_train, y_train)
# Best parameters
print("Best Parameters from Grid Search:\n", grid_search.best_params_)


# Best Model
best_mlp = grid_search.best_estimator_

# Predictions
y_pred = best_mlp.predict(X_test)
y_proba = best_mlp.predict_proba(X_test)[:, 1]

# Accuracy
print("\nTest Set Accuracy:", accuracy_score(y_test, y_pred))

# Classification Report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best Parameters from Grid Search:
 {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (200, 100, 100, 50, 50), 'learning_rate': 'constant', 'solver': 'adam'}

Test Set Accuracy: 0.9438202247191011

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.96      0.94        89
           1       0.95      0.93      0.94        89

    accuracy                           0.94       178
   macro avg       0.94      0.94      0.94       178
weighted avg       0.94      0.94      0.94       178



In [17]:
# Drop one feature from the dataset
features = merged_df.drop(["Sample", "disease_status"], axis=1)   
feature_names = features.columns.tolist()


In [18]:
feature_names

['DDX3Y',
 'ZCCHC12',
 'CNKSR2',
 'SLC14A1',
 'RBM3',
 'SERPINA5',
 'FRMPD4',
 'BEX1',
 'RPS4Y1',
 'PAK3',
 'BEX5',
 'FGF13',
 'GPRASP1',
 'SYTL4',
 'HPRT1',
 'TMSB4Y',
 'MS4A4A']

In [19]:
# Generate feature names automatically
feature_names = [f"Feature_{i+1}" for i in range(X_test.shape[1])]
# Convert NumPy array to DataFrame with these feature names
X = pd.DataFrame(X_test, columns=feature_names)
                 

In [20]:
import shap
import os

# ===== SHAP Analysis for MLP =====
# Create output directory
save_dir = "d:/Desktop/AD/AD_IQAC/IMAGE/"
os.makedirs(save_dir, exist_ok=True)

# Define prediction function (probabilities)
f = lambda x: best_mlp.predict_proba(x)

# Choose background (subset of training set to speed up KernelExplainer)
background = shap.sample(X_train, 100, random_state=42)

# Build explainer
explainer = shap.KernelExplainer(f, background)

# Compute SHAP values for test set (can be slow if large, so slice if needed)
print("Computing SHAP values (this may take time)...")
shap_values = explainer.shap_values(X)

# ===== Waterfall Plot for a single sample =====
sample_idx = 0   # first test sample
expl = shap.Explanation(
    values=shap_values[1][sample_idx],        # SHAP values for AD class
    base_values=explainer.expected_value[1],  # expected value for AD class
    data=X.iloc[sample_idx],             # one patient sample
    feature_names=feature_names
)

plt.figure(figsize=(10, 6))
shap.plots.waterfall(expl, max_display=10, show=False)
save_path = os.path.join(save_dir, "mlp_shap_waterfall_top10.png")
plt.savefig(save_path, bbox_inches='tight', dpi=300)
plt.close()
print(f"Waterfall plot saved at: {save_path}")


Computing SHAP values (this may take time)...


  0%|          | 0/178 [00:00<?, ?it/s]

Waterfall plot saved at: d:/Desktop/AD/AD_IQAC/IMAGE/mlp_shap_waterfall_top10.png
